### Plot read counts on chromosome X versus autosomes across 100 individuals chosen from the 1000 Genomes Project

In [39]:
# turn on inline plots
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [41]:
# Convinience definitions (data locations, etc)

baseDir='/scratch/biobootcamp_fluxod/kitzmanj/biobootcamp/material_for_flux'

fnSampleInfo = '%s/%s'%( baseDir, 'sample_info.txt' )
fnReadCounts = '%s/%s'%( baseDir, 'read_counts_by_indiv_and_chrom.txt' )

fnSampleList = '%s/%s'%( baseDir, 'sample_names.txt' )

#### open the table of per-sample information (sex, population, etc)

In [43]:
fileSampleInfo = open(fnSampleInfo,'r')

In [44]:
line = fileSampleInfo.readline()

In [16]:
line = line.rstrip()
line

'HG00099\tHG00099\tGBR\tBritish in England and Scotland\tfemale'

In [17]:
line = line.split('\t')
line

['HG00099', 'HG00099', 'GBR', 'British in England and Scotland', 'female']

In [19]:
sampleName = line[0]
print sampleName

HG00099


In [20]:
sampleSex = line[4]
print sampleSex

female


In [45]:
# a routine to parse the sample table, line-by-line, into a 
# dictionary mapping [individual name]-->[sex]

def parseSampleTable( filename ):
    fileSampTable = open(filename,'r')
    
    # skip past the file header
    fileSampTable.readline()

    sampleToSex={}
    
    # go line by line
    for line in fileSampTable:
        line = line.rstrip().split('\t')
        sampleToSex[ line[0] ] = line[4]
    
    return sampleToSex

In [46]:
sampToSex = parseSampleTable( fnSampleInfo )

In [48]:
#what is the sex of individual HG00099?
sampToSex[ 'HG00099' ]

'female'

#### DIY - what is the total number of males and females in this table?

In [51]:
n_male = 0
n_female = 0
for sample in sampToSex:
    if sampToSex[sample] == 'male':
        n_male+=1
    elif sampToSex[sample] == 'female':
        n_female+=1

print 'found ',n_male, 'males'
print 'found ',n_female, 'females'

found  1740 males
found  1760 females


###  Read count information

In [72]:
# e.g., let's look at one file
fnReadCounts = '%s/NA18627.stats.txt'%( baseDir )
filReadCounts = open(fnReadCounts,'r')
line = filReadCounts.readline()

In [73]:
line

'1\t249250621\t19054072\t64720\n'

In [74]:
line = line.rstrip().split('\t')
print line

['1', '249250621', '19054072', '64720']


### Routine to load the autosomal and chrX read count sums from one individual

In [87]:
def loadAutoAndXCounts( directory, sampName ):
    filename = '%s/%s.stats.txt'%( directory, sampName )
    fileReadCounts = open(filename,'r')
    
    lChromNamesAutosomes = [ '%d'%i for i in range(1,23) ]
    
    nAutosomalReads = 0
    nXReads = 0
    
    for line in fileReadCounts:
        line = line.rstrip().split('\t')
        if line[0] in lChromNamesAutosomes:
            nAutosomalReads += int( line[2] )
        elif line[0] == 'X':
            nXReads += int( line[2] )
        # line goes chromosome, chromosome size, number of mapped reads, number of unmapped reads
        
    return nAutosomalReads, nXReads

In [88]:
loadAutoAndXCounts( baseDir, 'NA18627')

(224463998, 12606641)

### Routine to load the each chromsome read counts from one individual

In [89]:
def loadPerChromCounts( directory, sampName ):
    filename = '%s/%s.stats.txt'%( directory, sampName )
    fileReadCounts = open(filename,'r')
    
    lChromosomes = []
    lReadCounts = [] 
    lChromSizes = []
        
    for line in fileReadCounts:
        line = line.rstrip().split('\t')
        lChromosomes.append( line[0] )
        lReadCounts.append( int(line[3]) )
        lChromSizes.append( int(line[1]) )
        
    return lChromosomes, lReadCounts, lChromSizes

In [91]:
lChromosomes,lReadCounts,lChromSizes = loadPerChromCounts( baseDir, 'NA18627')